# Modeling Agricultural Variables
## Python modules

In [59]:
import warnings
import time
import os
import random

import dask
from dask.distributed import Client

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from joblib import Parallel, delayed

import geopandas as gpd
import pyarrow

from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.metrics import mean_squared_error, confusion_matrix, r2_score, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr
from scipy.linalg import LinAlgWarning
from scipy.stats import pearsonr
from sklearn.utils import check_random_state, resample

import math
import seaborn as sns

## Read in Data

We first read in the aggregated features and ground-truth data joined in  feature_preprocessing.ipynb 

In [60]:
grouped_features = pd.read_csv("/capstone/mosaiks/repos/preprocessing/data/features_join_imputed.csv")

In [61]:
features = grouped_features.iloc[:,5:12005]
identifiers = grouped_features.iloc[:,0:5]
identifiers.head()

,year,sea_unq,index_left,lon,lat
0,2016.0,1,46302.000000,27.807993,-13.659357
1,2016.0,7,51611.666667,28.634660,-13.772690
2,2016.0,9,44806.714286,27.406446,-12.905428
3,2016.0,10,44644.411765,27.381719,-12.962298
4,2016.0,12,47769.000000,28.014660,-12.889357


In [62]:
outcomes = grouped_features.iloc[:,12006:]

outcomes["loss_ind"].astype('category')
outcomes["drought_loss_ind"].astype('category')
outcomes['pest_loss_ind'].astype('category')
outcomes['animal_loss_ind'].astype('category')
outcomes['flood_loss_ind'].astype('category')
outcomes.head()

print(outcomes.columns)

Index(['total_area_harv_ha', 'total_area_lost_ha', 'total_harv_kg',
       'yield_kgha', 'frac_area_harv', 'frac_area_loss', 'area_lost_fire',
       'maize', 'groundnuts', 'mixed_beans', 'popcorn', 'sorghum', 'soybeans',
       'sweet_potatoes', 'bunding', 'monocrop', 'mixture', 'frac_loss_drought',
       'frac_loss_flood', 'frac_loss_animal', 'frac_loss_pests',
       'frac_loss_soil', 'frac_loss_fert', 'prop_till_plough',
       'prop_till_ridge', 'prop_notill', 'prop_hand', 'prop_mono', 'prop_mix',
       'log_maize', 'log_sweetpotatoes', 'log_groundnuts', 'log_soybeans',
       'loss_ind', 'drought_loss_ind', 'flood_loss_ind', 'animal_loss_ind',
       'pest_loss_ind'],
      dtype='object')


## Model

We define a model to predict each of our outcome variables on our features for each SEA/year. The `train_and_evaluate_models` function trains and evaluates Ridge Linear Regression models for each target variable specified in the `target_columns` parameter. It handles both categorical and continuous target variables and provides the option to block sample on specific SEAs (Survey Enumeration Areas) by providing the SEA IDs to hold out for the validation set.

The function works as follows:

1. Read the grouped features and outcomes from a CSV file.
2. Define a helper function `block_sampling` to perform block sampling based on the provided SEA IDs.
3. For each target variable in `target_columns`, select the corresponding target variable data.
4. If `block_sea_ids` is provided and not empty, perform block sampling using the `block_sampling` helper function. Otherwise, use `train_test_split` to split the data into training and testing sets.
5. Train a Ridge Linear Regression model using RidgeCV with 5-fold cross-validation and a range of alpha values.
6. If the target variable is categorical, calculate and print the false positive rate and AUC-ROC. If the target variable is continuous, calculate and print the estimated regularization parameter, training R2 performance, validation R2 performance, and Pearson's correlation coefficient.

### Helper Function for Confusion Matrix for Categorical Variables
`calculate_confusion_matrix`:
This function calculates the confusion matrix for binary classification problems based on the given true labels (`y_true`), predicted values (`y_pred`), and a decision boundary (`decision_boundary`). The decision boundary is used to threshold the predicted values to obtain binary predictions.

Inputs:

`y_true`: The true labels of the target variable (a pandas Series or numpy array).

`y_pred`: The predicted values of the target variable (a numpy array).

`decision_boundary`: A float value that serves as the threshold for classifying the predicted values into two classes (0 or 1).


The function performs the following steps:
1. It adjusts the predicted values by setting them to 1 if they are greater than or equal to the decision boundary, and 0 otherwise.
2. It calculates the confusion matrix using the true labels and adjusted predicted values.
3. Depending on the shape of the confusion matrix, it extracts the true negatives (tn), false positives (fp), false negatives (fn), and true positives (tp).
4. If the shape of the confusion matrix is not (1, 1) or (2, 2), it raises an error.

Output: The function returns the values of tn, fp, fn, and tp.

In [63]:
def calculate_confusion_matrix(y_true, y_pred, decision_boundary):
    y_pred_adj = np.where(y_pred >= decision_boundary, 1, 0)
    cm = confusion_matrix(y_true, y_pred_adj)
    if cm.shape == (1, 1):
        if y_true.iloc[0] == 0:
            tn, fp, fn, tp = cm[0, 0], 0, 0, 0
        else:
            tn, fp, fn, tp = 0, 0, 0, cm[0, 0]
    elif cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
    else:
        print("Unexpected confusion matrix:")
        print(cm)
        raise ValueError('Unexpected confusion matrix shape.')
    return tn, fp, fn, tp

### Helper Function for Block Sampling on SEAs

This function randomly selects a specified number of unique SEA IDs from the `grouped_features` DataFrame.

Inputs:

1. n: The number of unique SEA IDs to select.

2. `grouped_features`: A DataFrame containing the feature data with a column '`sea_unq`' that stores the unique SEA IDs.

The function performs the following steps:

1. It extracts the unique SEA IDs from the '`sea_unq`' column of the `grouped_features` DataFrame.

2. It randomly selects n SEA IDs from the unique SEA IDs without replacement.

Output: The function returns a numpy array of the randomly selected SEA IDs.

These helper functions are used in the main model as follows:

`calculate_confusion_matrix` is used to calculate the confusion matrix for the categorical target variables. It is called in the `train_and_evaluate_models` function to compute the false positive rate and AUC-ROC for different decision boundaries.

`randomly_select_seas` is not used in the current implementation of the main model. However, it can be used to randomly select SEA IDs if you want to implement a custom sampling strategy.

In [64]:
def randomly_select_seas(n, grouped_features):
    unique_seas = grouped_features['sea_unq'].unique()
    selected_seas = np.random.choice(unique_seas, n, replace=False)
    return selected_seas

## Cross-Validator Custom Class

This custom cross-validator class, BlockSamplingCV, inherits from the BaseCrossValidator class in scikit-learn. It is designed to perform block sampling for cross-validation, holding out specific groups of observations (in this case, SEA IDs) during each split. This ensures that all observations with the same SEA ID are either in the training set or the test set, but not both.

Here's a detailed explanation of the class:

__init__(self, n_splits=5, n_seas_to_hold_out=10, sea_ids=None, random_state=None):
The constructor takes the following arguments:

n_splits: The number of cross-validation splits (default is 5).
n_seas_to_hold_out: The number of SEAs to hold out in each cross-validation split (default is 10).
sea_ids: A list or array of SEA IDs corresponding to the rows of the dataset (default is None).
random_state: An integer seed or a RandomState instance for reproducible results (default is None).
The constructor initializes the class with these arguments.

_iter_test_indices(self, X=None, y=None, groups=None):
This method generates test indices for each cross-validation split. It takes the following optional arguments:

X: Feature matrix (not used in this method but included for compatibility with scikit-learn).
y: Target variable (not used in this method but included for compatibility with scikit-learn).
groups: Group labels for the samples used to ensure that each group is either entirely in the training or test set (not used in this method but included for compatibility with scikit-learn).
The method performs the following steps:

a. It calculates the total number of samples and extracts the unique SEA IDs from the sea_ids attribute.
b. It initializes a random number generator with the specified random_state.
c. For each split, it randomly selects a set of n_seas_to_hold_out SEA IDs without replacement.
d. It finds the indices of the observations with the selected SEA IDs and yields them as test indices for the current split.

The BlockSamplingCV class is designed to perform cross-validation with block sampling, where groups of observations (in this case, SEA IDs) are held out together during each split. This is useful because it ensures that all observations with the same SEA ID are either in the training set or the test set, but not both. This can help prevent leakage of information between the training and test sets when observations with the same SEA ID are strongly correlated.

In [65]:
def get_train_val_test_indices(sea_ids, n_seas_to_hold_out_for_val, n_seas_to_hold_out_for_test, random_state):
    unique_seas = np.unique(sea_ids)
    np.random.seed(random_state)
    np.random.shuffle(unique_seas)

    # Hold out some SEAs for testing
    test_seas = unique_seas[:n_seas_to_hold_out_for_test]
    remaining_seas = unique_seas[n_seas_to_hold_out_for_test:]

    # Hold out some SEAs for validation
    val_seas = remaining_seas[:n_seas_to_hold_out_for_val]
    train_seas = remaining_seas[n_seas_to_hold_out_for_val:]

    # Convert boolean indices to integer indices
    train_indices = np.where(sea_ids.isin(train_seas))[0]
    val_indices = np.where(sea_ids.isin(val_seas))[0]
    test_indices = np.where(sea_ids.isin(test_seas))[0]

    return train_indices, val_indices, test_indices, train_seas, val_seas, test_seas


In [68]:
# Prepare the arguments as a dictionary
args = {
    'target_columns': ['yield_kgha', 'prop_till_plough', 'log_maize'],
    'test_size': 0.1,
    'categorical_columns':['loss_ind','drought_loss_ind', 'flood_loss_ind','animal_loss_ind','pest_loss_ind'],
    'decision_boundaries': [0.3,0.5,0.7],
    'sea_ids': grouped_features['sea_unq'],
    'validation_size' : 0.1,
    'bootstrap' : False,
    'n_bootstraps': None,
    'block_sample': False,
    'n_seas_held_out_val': None,
    'n_seas_held_out_test': None,
    'random_state': 50
}

In [69]:
def train_and_evaluate_models(args):
    target_columns = args['target_columns']
    test_size = args.get('test_size', 0.1)
    categorical_columns = args['categorical_columns']
    decision_boundaries = args['decision_boundaries']
    sea_ids = args['sea_ids']
    validation_size = args.get('validation_size', 0.1)
    bootstrap = args.get('bootstrap', False)
    n_bootstraps = args.get('n_bootstraps', 0)
    block_sample = args.get('block_sample', False)
    random_state = args.get('random_state', False)
    n_seas_held_out_val = args.get('n_seas_held_out_val', 30)
    n_seas_held_out_test = args.get('n_seas_held_out_test', 10)
    
    
    grouped_features = pd.read_csv("/capstone/mosaiks/repos/preprocessing/data/features_join_imputed.csv")

    lon = grouped_features.iloc[:,3]
    lat = grouped_features.iloc[:,4]
    year = grouped_features.iloc[:, 0]
    features = grouped_features.iloc[:, 5:12005]
    outcomes = grouped_features.iloc[:, 12006:]
    

    # Initialize an empty DataFrame to store the predictions
    train_predictions_df = pd.DataFrame()
    val_predictions_df = pd.DataFrame()
    # Initialize an empty DataFrame to store the metrics
    metrics_df = pd.DataFrame(columns=['target_column', 'train_score', 'val_score', 'pearson_coeff'])
    # Initialize an empty DataFrame to store groundtruth
    train_groundtruth_df = pd.DataFrame()
    val_groundtruth_df = pd.DataFrame()
    models = {}
    
    
    print(f"\nRunning model with the following parameters:")
    print(f"Target columns: {target_columns}")
    print(f"Test size: {test_size}", f"Validation size: {validation_size}")
    print(f"Bootstrap: {bootstrap}")
    print(f"Random State: {random_state}")
    if bootstrap:
        print(f"Number of bootstrapped samples: {n_bootstraps}")
    print(f"Block sample: {block_sample}\n")
    # If block_sample is True, then use block sampling
    if block_sample:
        # Get train, validation and test indices
        train_indices, val_indices, test_indices, train_seas, val_seas, test_seas = get_train_val_test_indices(sea_ids, n_seas_to_hold_out_for_val=n_seas_held_out_val, n_seas_to_hold_out_for_test=n_seas_held_out_test, random_state=random_state)
        print(f"Number of seas held out for validation: {n_seas_held_out_val}\n")
        print(f"Training SEAs: {train_seas}")
        print(f"Validation SEAs: {val_seas}")
        print(f"Testing SEAs: {test_seas}")
        

    for target_column in target_columns:
        # If block_sample is True, then use block sampling
        if block_sample:
            # Use the already obtained train_indices, val_indices, and test_indices
            # Create train, validation and test sets using the indices
            X_train, X_val, X_test = features.iloc[train_indices], features.iloc[val_indices], features.iloc[test_indices]
            y_train, y_val, y_test = outcomes[target_column].iloc[train_indices], outcomes[target_column].iloc[val_indices], outcomes[target_column].iloc[test_indices]

            X_train, X_val, X_test = X_train.iloc[:, 5:], X_val.iloc[:, 5:], X_test.iloc[:, 5:]
            
            val_groundtruth_df[target_column] = y_val
            train_groundtruth_df[target_column] = y_train
        
        
        elif bootstrap:
            # Split the data into training and test sets
            X_train, X_test, y_train, y_test = train_test_split(features, outcomes[target_column], test_size=test_size, random_state = random_state)
        
        else:
            X_train, X_test, y_train, y_test = train_test_split(features, outcomes[target_column], test_size=test_size, random_state = random_state)
            # Split the training data again to create a validation set
            X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size, random_state = random_state)
            
            val_groundtruth_df[target_column] = y_val
            train_groundtruth_df[target_column] = y_train
            
            sea_ids_val = [X_val.index]
            sea_ids_train = [X_train.index]

#         sea_ids_train = sea_ids[X_train.index]  # Update the sea_ids for the training set
#         sea_ids_test = sea_ids[X_test.index]  
        
        if bootstrap:
            # Define function for each bootstrap iteration
            def bootstrap_iteration(i):
                for i in range(n_bootstraps):
                    rs = i
                    # Set the random seed for reproducibility
                    rng = np.random.default_rng(rs)
                    # Create a bootstrapped training dataset
                    indices = rng.choice(len(X_train), size=len(X_train), replace=True)
                    X_train_bootstrap = X_train.iloc[indices]
                    y_train_bootstrap = y_train.iloc[indices]
                    X_train_bootstrap, X_val_bootstrap, y_train_bootstrap, y_val_bootstrap = train_test_split(X_train_bootstrap, y_train_bootstrap, test_size=validation_size, random_state=random_state)
                    
                    column_name = f"{target_column}_{i}"
                    val_groundtruth_df[column_name] = y_val_bootstrap
                    train_groundtruth_df[column_name] = y_train_bootstrap
                
                    # Fit the model to the bootstrapped data
                    cv = 5
                    ridge_cv = RidgeCV(cv=cv, alphas=np.logspace(-8, 8, base=10, num=75))
                    ridge_cv.fit(X_train_bootstrap, y_train_bootstrap)
                    
                    models[column_name] = ridge_cv
                    
                    # Store the coefficients and validation scores
                    train_score = ridge_cv.score(X_train_bootstrap, y_train_bootstrap)
                    val_score = r2_score(y_val_bootstrap, ridge_cv.predict(X_val_bootstrap))
                    pearson_coeff, _ = pearsonr(y_val_bootstrap, ridge_cv.predict(X_val_bootstrap))
                    
                    return ridge_cv.coef_, train_score, val_score, pearson_coeff, val_groundtruth_df, train_groundtruth_df, models
            
            
            # Run bootstrap iterations in parallel
            results = Parallel(n_jobs=-1)(delayed(bootstrap_iteration)(i) for i in range(n_bootstraps))
            # Unpack results
            coefs, train_scores, val_scores, pearson_coeffs, *df_columns, models = zip(*results)
            
            val_groundtruth_df = pd.DataFrame(dict(zip(df_columns, zip(*val_groundtruth_df))))
            train_groundtruth_df = pd.DataFrame(dict(zip(df_columns, zip(*train_groundtruth_df))))

            # Calculate the average coefficients and validation scores
            val_groundtruth_df = val_groundtruth_df.filter(like=target_column).mean(axis=1)
            train_groundtruth_df = train_groundtruth_df.filter(like=target_column).mean(axis=1)
            avg_coefs = np.mean(coefs, axis=0)
            avg_train_score = np.mean(train_scores)
            avg_val_score = np.mean(val_scores)
            avg_pearson_coeff = np.mean(pearson_coeffs)
            
            print(f"Target variable: {target_column}")
            print(f"Average training R2 score: {avg_train_score:0.2f}")
            print(f"Average validation R2 score: {avg_val_score:0.2f}")
            print(f"Average Pearson’s correlation coefficient: {avg_pearson_coeff:0.2f}")
            
            if target_column in categorical_columns:
                print(f"Average false positive rate: {avg_false_positive_rate:0.2f}")
                print(f"Average AUC-ROC: {avg_auc_roc:0.2f}")
            print()

        else:
            cv = 5
            ridge_cv = RidgeCV(cv=cv, alphas=np.logspace(-8, 8, base=10, num=75))
            ridge_cv.fit(X_train, y_train)
            
            models[target_column] = ridge_cv
            

            # Make predictions on the test data
            y_val_pred = ridge_cv.predict(X_val)
            y_train_pred = ridge_cv.predict(X_train)
    
            # Update the predictions DataFrame with the new predictions
            val_predictions_df[target_column] = y_val_pred
            train_predictions_df[target_column] = y_train_pred
    
            if target_column in categorical_columns:
                for decision_boundary in decision_boundaries:
                    # Calculate confusion matrix
                    tn, fp, fn, tp = calculate_confusion_matrix(y_val, y_val_pred, decision_boundary)

                # Calculate the false positive rate
                    false_positive_rate = fp / (fp + tn)

                # Calculate AUC-ROC
                    auc_roc = roc_auc_score(y_val, y_val_pred)

                    print(f"Target variable: {target_column} (Categorical)")
                    print(f"Decision boundary: {decision_boundary}")
                    print(f"False positive rate: {false_positive_rate:0.2f}")
                    print(f"AUC-ROC: {auc_roc:0.2f}")
                    print()
            else:
                # Calculate Pearson's correlation coefficient
                pearson_coeff, _ = pearsonr(y_val, y_val_pred)

                # Calculate training R squared
                train_r_squared = ridge_cv.score(X_train, y_train)

                # Calculate validation R squared
                val_r_squared = ridge_cv.score(X_val, y_val)
                
                metrics_df = metrics_df.append({
                'target_column': target_column,
                'train_score': train_r_squared,
                'val_score': val_r_squared,
                'pearson_coeff': pearson_coeff}, ignore_index=True)
                    
                print()
                print(f"Target variable: {target_column}")
                print(f"Estimated regularization parameter: {ridge_cv.alpha_}")
                print(f"Training R2 performance: {train_r_squared:0.2f}")
                print(f"Validation R2 performance: {val_r_squared:0.2f}")
                print(f"Pearson's correlation coefficient: {pearson_coeff:0.2f}")
                print()
    
    val_predictions_df['lat'] = lat.iloc[X_val.index].values.flatten()
    val_predictions_df['lon'] = lon.iloc[X_val.index].values.flatten()
    val_predictions_df['year'] = year.iloc[X_val.index].values.flatten()
    train_predictions_df['lat'] = lat.iloc[X_train.index].values.flatten()
    train_predictions_df['lon'] = lon.iloc[X_train.index].values.flatten()
    train_predictions_df['year'] = year.iloc[X_train.index].values.flatten()

    return val_predictions_df, train_predictions_df, metrics_df, val_groundtruth_df, train_groundtruth_df, models 

In [70]:
val_predictions_df, train_predictions_df, metrics_df, val_groundtruth_df, train_groundtruth_df, models  = train_and_evaluate_models(args)


Running model with the following parameters:
Target columns: ['yield_kgha', 'prop_till_plough', 'log_maize']
Test size: 0.1 Validation size: 0.1
Bootstrap: False
Random State: 50
Block sample: False



/tmp/ipykernel_705280/2197089475.py:186: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({



Target variable: yield_kgha
Estimated regularization parameter: 1.6451905877536674
Training R2 performance: 0.74
Validation R2 performance: 0.62
Pearson's correlation coefficient: 0.80



/tmp/ipykernel_705280/2197089475.py:186: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({



Target variable: prop_till_plough
Estimated regularization parameter: 7.3259654282152304
Training R2 performance: 0.78
Validation R2 performance: 0.71
Pearson's correlation coefficient: 0.85


Target variable: log_maize
Estimated regularization parameter: 1.6451905877536674
Training R2 performance: 0.77
Validation R2 performance: 0.71
Pearson's correlation coefficient: 0.84



/tmp/ipykernel_705280/2197089475.py:186: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


In [71]:
final_val_preds = pd.merge(val_predictions_df, identifiers, on=['lat','lon','year'], how = 'left')

In [72]:
val_predictions_df.head()

,yield_kgha,prop_till_plough,log_maize,lat,lon,year
0,1092.153751,-0.014105,5.873944,-13.806253,26.563983,2016.0
1,1468.369828,0.274426,3.704600,-15.431055,26.053528,2017.0
2,-351.338579,0.840106,3.071214,-16.703643,26.734660,2016.0
3,1847.192846,0.431688,6.629293,-16.753474,27.525248,2021.0
4,1299.275869,0.373903,5.384790,-11.178448,33.279660,2017.0


In [73]:
train_predictions_df.head()

,yield_kgha,prop_till_plough,log_maize,lat,lon,year
0,156.162266,0.294333,3.209652,-9.901757,32.465860,2017.0
1,38.363936,0.300490,3.106022,-12.145131,25.050857,2017.0
2,1373.119866,0.903735,6.483269,-16.226857,26.948410,2020.0
3,2307.667465,0.525004,6.507477,-13.276857,24.587160,2021.0
4,2429.729313,0.184509,8.397025,-12.910168,27.114119,2020.0


In [74]:
all_training_predictions = pd.concat([val_predictions_df, train_predictions_df], axis=0)

In [75]:
all_training_predictions.head()

,yield_kgha,prop_till_plough,log_maize,lat,lon,year
0,1092.153751,-0.014105,5.873944,-13.806253,26.563983,2016.0
1,1468.369828,0.274426,3.704600,-15.431055,26.053528,2017.0
2,-351.338579,0.840106,3.071214,-16.703643,26.734660,2016.0
3,1847.192846,0.431688,6.629293,-16.753474,27.525248,2021.0
4,1299.275869,0.373903,5.384790,-11.178448,33.279660,2017.0


In [76]:
all_training_predictions.to_csv('/capstone/mosaiks/repos/modeling/presentation_viz/train_preds_sea_cv.csv', index=False)

KeyError: 1

In [ ]:
if __name__ == "__main__":
    # Set the random seed for reproducibility
    random.seed(42)

    # Define the number of splits
    n_splits = 10

    # Generate n_splits random integers between 0 and 1,000,000
    random_seeds = [random.randint(0, 1_000_000) for _ in range(n_splits)]

    # Initialize an empty list to store the prediction DataFrames
    all_predictions = []

    # Define the arguments for each call to train_and_evaluate_models
    args = {
        # Add your arguments here
    }

    # Loop over the random seeds
    for random_state in random_seeds:
        # Update the random state in the arguments
        args['random_state'] = random_state

        # Call train_and_evaluate_models and store the predictions
        predictions_df = train_and_evaluate_models(args)

        # Append the predictions DataFrame to the list
        all_predictions.append(predictions_df)

    # At this point, all_predictions is a list of prediction DataFrames, one for each split.


#### 

In [80]:
zambia = pd.read_csv("/capstone/mosaiks/repos/modeling/data/zambia_10perc_imputed_2023_05_16.csv")

In [93]:
identifiers = zambia.iloc[:,0:5]
identifiers = identifiers.drop(columns=['index_left'])
zambia_features = zambia.iloc[:,5:]

,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,0_10,...,999_3,999_4,999_5,999_6,999_7,999_8,999_9,999_10,999_11,999_12
0,0.001077,0.0007,0.000706,0.000395,0.000172,0.000183,0.000276,0.000605,0.002596,0.001407,...,0.058021,0.075707,0.056214,0.056737,0.036262,0.004133,0.000831,0.002931,0.004228,0.03583
1,0.001077,0.0007,0.000706,0.000395,0.000172,0.000183,0.000276,0.000605,0.001739,0.001407,...,0.058021,0.075707,0.056214,0.056737,0.036262,0.004133,0.000000,0.002931,0.004228,0.03583
2,0.001077,0.0007,0.000706,0.000395,0.000172,0.000183,0.000276,0.000605,0.005177,0.001407,...,0.058021,0.075707,0.056214,0.056737,0.036262,0.004133,0.000186,0.002931,0.004228,0.03583
3,0.001077,0.0007,0.000706,0.000395,0.000172,0.000183,0.000276,0.000605,0.003367,0.001407,...,0.058021,0.075707,0.056214,0.056737,0.036262,0.004133,0.000870,0.002931,0.004228,0.03583
4,0.001077,0.0007,0.000706,0.000395,0.000172,0.000183,0.000276,0.000605,0.003582,0.001407,...,0.058021,0.075707,0.056214,0.056737,0.036262,0.004133,0.000792,0.002931,0.004228,0.03583


In [94]:
identifiers

,year,ward_id,lon,lat
0,2015.0,0,22.799338,-15.095757
1,2015.0,1,22.660588,-14.993257
2,2015.0,2,22.465033,-15.005479
3,2015.0,3,22.559819,-15.041718
4,2015.0,4,22.718088,-15.097007
...,...,...,...,...
10986,2023.0,1409,28.359715,-14.934616
10987,2023.0,1410,30.348366,-15.581035
10988,2023.0,1412,28.704943,-9.254870
10989,2023.0,1413,29.503497,-9.768893


In [82]:
zambia_predictions_df = pd.DataFrame()

for target_column, model in models.items():
    predictions = model.predict(zambia_features)
    zambia_predictions_df[target_column] = predictions

In [95]:
zambia_predictions_df.head()
zambia_predictions = pd.merge(identifiers, zambia_predictions_df, left_index=True, right_index=True)

In [98]:
zambia_predictions.to_csv('/capstone/mosaiks/repos/modeling/presentation_viz/train_preds_zambia_cv.csv', index=False)

### Train set

In [ ]:
y_pred = np.maximum(ridge_cv_random.predict(x_train), 0)
r2_train = r2_score(y_train, y_pred)

fig, ax = plt.subplots(ncols=1)
plt.scatter(y_pred, y_train, alpha=1, s=4)
plt.xlabel("Predicted", fontsize=15, x = .3)
plt.ylabel("Ground Truth", fontsize=15)
plt.suptitle(r"$\log_{10}(1 + Crop Yield)$", fontsize=20, y=1.02)
plt.title((f"Model applied to train data n = {len(x_train)}, R$^2$ = {r2_train:0.2f}"),
          fontsize=12, y=1.01)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

ax.axline([0, 0], [1, 1], c = "k")

plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)


# plt.savefig(f'images/{feature_file_name}_train_data.jpg', dpi=300)
plt.show()
plt.close()
# the model is plotted with a black 45 degree line that serves as a reference of what a perfect correlation would look like
# deviation of the line indicates that there is not a perfect correlation

In [ ]:
print(f"Training R^2 = {r2_train:0.2f}\nPearsons r = {pearsonr(y_pred, y_train)[0]:0.2f}") 

In [ ]:
# Pearson r^2
pearsonr(y_pred, y_train)[0] ** 2

In [ ]:
# alternative way to calculate Training R^2
ridge_cv_random.score(x_train, y_train)

### Test set

In [ ]:
y_pred = np.maximum(ridge_cv_random.predict(x_test), 0)
r2_test = r2_score(y_test, y_pred)

plt.figure()
plt.scatter(y_pred, y_test, alpha=1, s=4)
plt.xlabel("Predicted", fontsize=15)
plt.ylabel("Ground Truth", fontsize=15)
plt.suptitle(r"$\log_{10}(1 + Crop Yield)$", fontsize=20, y=1.02)
plt.title(f"Model applied to test data n = {len(x_test)}, R$^2$ = {r2_test:0.2f}",
          fontsize=12, y=1)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

ax.axline([0, 0], [.75, .75], c = "k")

plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)

# plt.savefig(f'images/{feature_file_name}_test_data.jpg', dpi=300)
plt.show()
plt.close()

In [ ]:
print(f"Testing set R^2 = {r2_test:0.2f}")
print(f"Testing set pearsons R = {pearsonr(y_pred, y_test)[0]:0.2f}")

Summary of both train and test data sets

In [ ]:
y_pred = np.maximum(ridge_cv_random.predict(x_all), 0)

fig, ax = plt.subplots(figsize=(7, 7))
ax.axline([0, 0], [.75, .75], c = "k")
plt.scatter(y_pred, y_all, alpha=.9, s=15)
plt.xlabel("Predicted", fontsize=15)
plt.ylabel("Observed", fontsize=15)
plt.text(
    0, .8, fontsize=15, fontweight="bold",
    s=f"R$^2$={r2_train:0.2f} - Train set",
)
plt.text(
    0, .75, fontsize=15, fontweight="bold",
    s=f"R$^2$={ridge_cv_random.best_score_:0.2f} - Validation set",
)
plt.text(
    0, .7, fontsize=15, fontweight="bold",
    s=f"R$^2$={r2_test:0.2f} - Test set",
)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)

# plt.savefig(f'images/{feature_file_name}_all_data.jpg', dpi=300)
plt.show()
plt.close()

### Use the trained model to predict crop yields over all years from 1km grid-cell resolution features 

Recall that after we executed imputation on all feature years in the dataframe `features`, we copied the dataframe and named it `features_all_years`. Now we can plug that into the model to visualize how our model performs over time.

In [ ]:
# recall the object we created earlier, before we split the features by year into those that would train the model 
# and those that would be fed into the trained model to predict crop yields
# in years for which we do not have crop data
features_all_years.head(3)

In the following chunk, we drop certain columns from `features_all_years` because we only need to feed the feature data into the model to generate predictions. Using the argument `axis = 1`, we specify that we are dropping columns rather than rows. 

In [ ]:
x_all = features_all_years.drop([
    'year', 
    'geometry',
    'district',
    'crop_perc'
], axis = 1)

In the following chunk, we execute the model on the features from the dataframe `features_all_years`. The crop yield predictions for each row populate a new column in the dataframe.

The model is run inside the `np.maximum()` function because if we run it without being wrapped inside function, some crop predictions are negative values, but we need them all to be positive because conceptually crop yields cannot be negative.

In [ ]:
features_all_years['yield_prediction'] = np.maximum(ridge_cv_random.predict(x_all), 0)

In [ ]:
# check out the dataframe with the new column of predictions
features_all_years.head(3)

The dataframe is already a geodataframe, so we do not have to convert it to one before mapping predictions. However, we do need to replace all the zero value crop percentage areas with `NA`. We do this by applying the `mask()` function. This function is similar to an if-else statement. If the value of the `crop_perc` is equal to 0, that value is replaced by the value of the second argument, which is `NA`. If the value of `crop_prec` is _not_ equal to zero, we retain the current value. The argument `inplace = True` executes this replacement in the same cell. 

In [ ]:
features_all_years['yield_prediction'].mask(features_all_years['crop_perc']==0, np.nan, inplace=True)

Recall that this dataframe has a geometry column, with latitude and longitude together. In order to map the predicted features, we separate this geometry column into separate `lon` and `lat` columns. 

In [ ]:
# extract the longitude and latitude from the geometry column, and make then into independent columns
features_all_years['lon'], features_all_years['lat'] = features_all_years.geometry.x, features_all_years.geometry.y

Plot the predicted features for each year:

In [ ]:
def scatter(x, y, c, **kwargs):
    plt.scatter(x, y, c=c, s = 1.25)
sns.color_palette("viridis", as_cmap=True)
g = sns.FacetGrid(
    features_all_years, 
    col="year", 
    col_wrap = 4, 
    height=5, 
    aspect=1
)
g.map(scatter, "lon", "lat", "yield_prediction")
g.set_axis_labels(r"Yield Prediction")
# save the figure and name the file so that it represents the model parameters that created the predictions
# plt.savefig(f'images/{feature_file_name}_all_predictions.jpg', dpi=300)

Plot the model's predicted features summarized to district level. In this visualization, we choose a specific year to examine rather than visualizing all years in one figure. Visualizing the the features summarized to district level is interesting because the crop data resolution provided by Zambia Statistics Agency is at the district level, and therefore it is easier to compare our model results to those ground-truth values when they are summarized to district level as well. Furthermore, our model's crop predictions for the years 2020 and 2021 might be more valuable when summarized to district level if Zambian governments, policy-makers, farmers, and researchers wish to use this data to determine crop imports, exports, and storage according to district summaries. 

In [ ]:
features_all_years_summary = (
    features_all_years
    .groupby(['district',"year"], as_index = False)['yield_prediction']
    .mean()
    .set_index('district')
)

In [ ]:
# join Zambia's shapefile to the summarized features to map the districts
# reset the index so it is a properly formatted dataframe
features_all_years_summary = features_all_years_summary.join(country_shp).reset_index()

Now that the geometries have been converted to districts from points, the geomatries are now polygons. There is still a row for each district for each year.

In order to change the year visualized, simply change the year in the following code and re-run the chunk.

In [ ]:
features_all_years_summary[features_all_years_summary.year == 2020].plot(column = "yield_prediction")

Plot a boxplot for each year to visualize the range and quantile distribution of each year's crop predictions, summarized to district level. This enables us to identify years with exceptional disparities between the predicted yields by district. It also allows us to identify years that have many outliers.

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x="year", y="yield_prediction", data = features_all_years_summary)
plt.xlabel("Year", fontsize=15)
plt.ylabel("Predicted Yield", fontsize=15)

Visualize the total crop yield predictions by year. This bar chart shows the sum of all the district crop yields.

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x="year", y="yield_prediction", data = features_all_years_summary, estimator = sum)

## Yield and Residual Plots

Create a dataframe of residuals called `residuals_df` from the `features_summary` dataframe. Note that we are _not_ using the predicted crop yields for _all_ years for these residuals, but rather the ground-truth crop yields for just the years through 2018.

The residuals give us an idea of the amount of uncertianty that is present in our model. By demeaning the residuals over space, we are able to remove the uncertainty over space and better determine our model performance over time and our uncertainty over time.

In [ ]:
x_all = features_summary.drop(drop_cols, axis = 1)

# create empty dataframe to then populate with columns
residual_df = pd.DataFrame()

residual_df["yield_mt"] = features_summary.yield_mt.to_numpy()
residual_df["log_yield"] = np.log10(features_summary.yield_mt.to_numpy() + 1)
residual_df["prediction"] = np.maximum(ridge_cv_random.predict(x_all), 0)
residual_df["residual"] = residual_df["log_yield"] - residual_df["prediction"]
residual_df["year"] = features_summary.year
residual_df["district"] = features_summary.district
# join the district geometries
residual_df = residual_df.join(country_shp, how = "left", on = "district")

# demean by location so we can analyze the data over time
residual_df["district_yield_mean"] = residual_df.groupby('district')['log_yield'].transform('mean')
residual_df["district_prediction_mean"] = residual_df.groupby('district')['prediction'].transform('mean')
residual_df["demean_yield"] = residual_df["log_yield"] - residual_df["district_yield_mean"]
residual_df["demean_prediction"] = residual_df["prediction"] - residual_df["district_prediction_mean"]
residual_gdf = geopandas.GeoDataFrame(residual_df)

residual_gdf.head(3)

Visualize the residuals for the ground truth crop yields through 2018 with a boxplot.

In [ ]:
plt.figure(figsize=(6, 5))
sns.boxplot(x="year", y="log_yield", data=residual_df)
plt.xlabel("Year", fontsize=15)
plt.ylabel("Log Yield", fontsize=15)

Visualize the residuals as a sum by year with a bar plot.

In [ ]:
plt.figure(figsize=(6, 5))
sns.barplot(x="year", y="log_yield", data=residual_df, estimator = sum)

Visualize the crop yield residuals by year as a histogram to determine how they are distributed.

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.histplot, "yield_mt", bins = 20)
g.set_axis_labels("Yield (MT)")

Visualize the log-transformed crop yield residuals by year as a histogram to compare how they are distributed after the transformation.

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.histplot, "log_yield", bins = 20)
g.set_axis_labels(r"$\log_{10}(1 + Crop Yield)$")

#### Crop prediction histogram

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.histplot, "prediction", bins = 20)
g.set_axis_labels(r"Crop yield predictions")

#### Residual histogram

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.histplot, "residual", bins = 20)
g.set_axis_labels(r"Residuals")

In [ ]:
residual_gdf.residual.min()

In [ ]:
residual_gdf.residual.max()

#### Log crop yield vs residuals

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.scatterplot, "log_yield", "residual")
g.set_axis_labels(r"$\log_{10}(1 + Crop Yield)$")

#### District residuals 

In [ ]:
if satellite == 'landsat-8-c2-l2':
    fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(13, 5))
    ax1 = (residual_gdf[residual_gdf.year == 2014]
           .plot(ax = ax1, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
           .set_title("2014 Residuals"))
    ax2 = (residual_gdf[residual_gdf.year == 2015]
           .plot(ax = ax2, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
           .set_title("2015 Residuals"))
else:
    pass
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
ax1 = (residual_gdf[residual_gdf.year == 2016]
       .plot(ax = ax1, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
       .set_title("2016 Residuals"))
ax2 = (residual_gdf[residual_gdf.year == 2017]
       .plot(ax = ax2, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
       .set_title("2017 Residuals"))
ax3 = (residual_gdf[residual_gdf.year == 2018]
       .plot(ax = ax3, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
       .set_title("2018 Residuals"))

caption = "A positive value is an underestimated prediction (the prediction is lower than the actual yield), a negative value is an over estimated prediction"
plt.figtext(0.5, 0.01, caption, wrap=True, horizontalalignment='center', fontsize=12)


#### Difference from the mean

In [ ]:
g = sns.FacetGrid(
    residual_gdf, 
    col="year", 
#     col_wrap = 3, 
    height=4, 
    aspect=1
)
g.map(sns.scatterplot, "demean_yield", "demean_prediction")
g.set_axis_labels('Difference from Yield Mean', 'Difference from Prediction Mean')

In [ ]:
fig, ax = plt.subplots(figsize= (6, 5))
ax.axline([-.2, -.2], [.2, .2], c = "k")
plt.scatter(residual_gdf.demean_yield, residual_gdf.demean_prediction)
plt.title("Demeaned truth and predictions by district")
plt.xlabel('Difference from Yield Mean')
plt.ylabel('Difference from Predictions Mean')
r_squared = r2_score(residual_gdf["demean_yield"], residual_gdf["demean_prediction"])
plt.text(
    -0.2,
    .18,
    s=f"Demeaned R$^2$ = {r_squared:0.2f}",
    fontsize=15,
    fontweight="bold",
)
plt.savefig(f'images/{feature_file_name}_demean.jpg', dpi=300)

In [ ]:
for yr in range(year_start+1, 2018):
    r_squared = r2_score(residual_gdf[residual_gdf.year == yr]["demean_yield"], residual_gdf[residual_gdf.year == yr]["demean_prediction"])
    pearson_r = pearsonr(residual_gdf[residual_gdf.year == yr]["demean_yield"], residual_gdf[residual_gdf.year == yr]["demean_prediction"])
    
    print(yr, f"    R^2: {r_squared:.2f}\n",
          f"Pearson's r: {pearson_r[0]:.2f}\n", 
          sep = "")
    
r_squared = r2_score(residual_gdf["demean_yield"], residual_gdf["demean_prediction"])
pearson_r = pearsonr(residual_gdf["demean_yield"], residual_gdf["demean_prediction"])
print(f"All     R^2: {r_squared:.2f}\n",
      f"Pearson's r: {pearson_r[0]:.2f}", sep = "")

In [ ]:
r2 = round(pearson_r[0] ** 2, 2)
r2

#### Join residuals to the features for _all_ years to visualize the residuals of the features before they were summarized to district level.

In [ ]:
complete_df = (
    features_all_years_summary
    .set_index(['district', 'year'])
    .join(residual_df
          .drop('geometry', axis = 1)
          .set_index(['district', 'year'])
         )
    .reset_index()
)

complete_df.head(3)

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
tidy = complete_df.melt(id_vars='year').rename(columns=str.title)
tidy = tidy[tidy.Variable.isin(['yield_prediction', 'log_yield'])]
sns.barplot(x='Year', y='Value', hue='Variable', data=tidy, ax=ax1, ci = None)
sns.despine(fig)

h, l = ax1.get_legend_handles_labels()
ax1.legend(h, ['Predicted Yield', 'Observed Yield'],loc='lower left')

plt.savefig(f'images/{feature_file_name}_yield_pred.jpg', dpi=300)

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x="year", y="yield_prediction", data=complete_df, estimator = sum)

### Congratulations on completing this analysis!